# LangChain - Retrieval Augmented Generation

## Abstract

In this notebook, we demonstrate LangChain with an example involving RAG, input/output parsing, output formatting, etc.

**RAG-based Question-Answering System using LangChain**

In this example, we use a web page as our knowledge source and build a chain that can answer questions about its content.

* Data Source – https://lilianweng.github.io/posts/2023-06-23-agent/​

* LLM –  Gpt-4o-mini ​

* Vector Store – ChromaDB​

In [1]:
from openai import OpenAI
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
with open('api_key.txt') as ap:
    api_key = ap.read()
#     print(api_key)

### Load Data and Create Vector Store

In [3]:
client = OpenAI(api_key=api_key)

# Load the web page
loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
docs = loader.load()

# Split the document into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

# Vector store
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings(api_key=api_key))

/var/folders/hq/dgkwpqzd10xgjv9hflydy7tm0000gn/T/ipykernel_9917/1523640762.py:12: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings(api_key=api_key))


### Setup Retriever and RAG Chain

In [4]:
retriever = vectorstore.as_retriever()

# Custom prompt template
template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer in the following format:
Answer: [Your answer here]
Source: [Page title or URL of the source]
Confidence: [High/Medium/Low]
"""

prompt = ChatPromptTemplate.from_template(template)

# RAG chain
model = ChatOpenAI(api_key=api_key, model_name="gpt-4o-mini", temperature=0)
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

/var/folders/hq/dgkwpqzd10xgjv9hflydy7tm0000gn/T/ipykernel_9917/3461173783.py:18: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  model = ChatOpenAI(api_key=api_key, model_name="gpt-4o-mini", temperature=0.3)


### Custom Output Parser

In [5]:
from langchain.schema import BaseOutputParser

class CustomOutputParser(BaseOutputParser):
    def parse(self, text):
        lines = text.strip().split("\n")
        result = {}
        for line in lines:
            if ": " in line:
                key, value = line.split(": ", 1)
                result[key.lower()] = value.strip()
            else:
                # Handle cases where there's no colon separator
                result['content'] = line.strip()
        return result
    
output_parser = CustomOutputParser()


### Final Chain

In [6]:
from langchain.schema import StrOutputParser

def format_output(parsed_output):
    return f"""
## Question Answered
Answer: {parsed_output['answer']}

**Source:** {parsed_output['source']}
**Confidence:** {parsed_output['confidence']}
"""

final_chain = rag_chain | output_parser | format_output

### Query and Response

In [8]:
question = "What is task decomposition in the context of AI agents?"
result = final_chain.invoke(question)
print(f"##Question: \n {question}")
print(result)

##Question: 
 What is task decomposition in the context of AI agents?

## Question Answered
Answer: Task decomposition in the context of AI agents refers to the process of breaking down complex tasks into smaller, manageable subgoals. This allows the agent to handle complicated tasks more efficiently. Techniques such as Chain of Thought (CoT) prompting are used to guide the model in thinking step by step, transforming larger tasks into simpler ones, and providing insights into the model's reasoning process. Additionally, task decomposition can be achieved through simple prompting, task-specific instructions, or human inputs.

**Source:** LLM Powered Autonomous Agents | Lil'Log
**Confidence:** High



In [9]:
question = "Explain self-reflection in the context of AI agents?"
result = final_chain.invoke(question)
print(f"##Question: \n {question}")
print(result)

##Question: 
 Explain self-reflection in the context of AI agents?

## Question Answered
Answer: Self-reflection in the context of AI agents is a vital aspect that allows these agents to improve iteratively by refining past action decisions and correcting previous mistakes. It involves the agent analyzing its past actions, learning from failures, and making adjustments for future tasks. This process is crucial in real-world scenarios where trial and error are common, enabling the agent to enhance its performance over time. In frameworks like Reflexion, self-reflection is facilitated by showing the agent examples of failed trajectories alongside ideal reflections, which are then stored in the agent's working memory to guide future actions.

**Source:** LLM Powered Autonomous Agents | Lil'Log (https://lilianweng.github.io/posts/2023-06-23-agent/)
**Confidence:** High

